In [0]:
!pip install -q keras

In [1]:
import keras
from google.colab import drive

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive', force_remount =True)

In [0]:
!pip show keras

## Prediction Models

In [10]:
from keras.applications import vgg16, vgg19, resnet50, inception_v3, inception_resnet_v2, xception, densenet, nasnet

# #224x224 input models
# #Load the VGG model
vgg16_model = vgg16.VGG16(weights='imagenet')

# #Load the VGG model
vgg19_model = vgg19.VGG19(weights='imagenet')

# #Load the VGG model
resnet50_model = resnet50.ResNet50(weights='imagenet')

# #Load the VGG model
densenet121_model = densenet.DenseNet121(weights='imagenet')

# #Load the VGG model
densenet169_model = densenet.DenseNet169(weights='imagenet')

# #Load the VGG model
densenet201_model = densenet.DenseNet201(weights='imagenet')

##----------------------------------------##

#299x299 input models
#Load the Inception_V3 model 
inceptionv3_model = inception_v3.InceptionV3(weights='imagenet') 

#Load the InceptionResNetV2 model
inception_resnet_v2_model = inception_resnet_v2.InceptionResNetV2(weights='imagenet')

#Load the Xception model
xception_model = xception.Xception(weights='imagenet')

##----------------------------------------##

#Need 331x331
#Load the NASNet model 
nasnet_model = nasnet.NASNetLarge(weights='imagenet') 


Instructions for updating:
Colocations handled automatically by placer.
359751680/359748576 [==============================] - 6s 0us/step


## Load Labels

In [0]:
import numpy as np
from keras.utils import to_categorical

In [0]:
y_val = np.load("/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/y_val.npy")

In [0]:
#Do this once!
#Make rows in y_val a multiple of 3500; coz we have 3500 rows
# y_val = np.append (y_val, np.zeros(2500)) 

y_val = y_val.reshape((15, 3500 ))
y_val_one_hot = to_categorical(y_val[0], 1000)

In [0]:
keras_idx_to_name = {}
f = open("/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()  

## BenchMarking Logic

In [0]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [0]:
def savePredictions(m_i, val):
  base_path = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetBenchMarkingResults/Predictions/"
  np.save(base_path + str(m_i) + ".npy", val)
  
def loadPredictions(m_i):
  base_path = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetBenchMarkingResults/Predictions/"
  return np.load(base_path + str(m_i) + ".npy")

In [0]:
def preprocessData(x_val, mClass):
  if mClass == 0:
    x_val = vgg16.preprocess_input(x_val) # converted to BGR
  elif mClass == 1:
    x_val = vgg19.preprocess_input(x_val) # converted to BGR
  elif mClass == 2:
    x_val = resnet50.preprocess_input(x_val) # converted to BGR
  elif mClass == 3:
    x_val = densenet.preprocess_input(x_val) # converted to BGR
  elif mClass == 4:
    x_val = densenet.preprocess_input(x_val) # converted to BGR
  elif mClass == 5:
    x_val = densenet.preprocess_input(x_val) # converted to BGR
  elif mClass == 6:
    x_val = inception_v3.preprocess_input(x_val) # converted to BGR
  elif mClass == 7:
    x_val = inception_resnet_v2.preprocess_input(x_val) # converted to BGR
  elif mClass == 8:
    x_val = xception.preprocess_input(x_val) # converted to BGR
  elif mClass == 9:
    x_val = nasnet.preprocess_input(x_val) # converted to BGR
  return x_val

def savePredictions(predictionModels, imageRes = "224x224", base = 0):
  
  # imageRes = "224x224"
  # imageRes = "299x299"
  # imageRes = "331x331"

  X = ["" for x in range(7)]
  X[0] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_1.npy"
  X[1] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_2.npy"
  X[2] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_3.npy"
  X[3] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_4.npy"
  X[4] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_5.npy"
  X[5] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_6.npy"
  X[6] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_7.npy"
  #X[7] = "/content/drive/My Drive/MS_CS_DS/Sem2/AI/ImageNetValidationMetaDeta/" + imageRes + "/x_val_8.npy"
  

  totalModels = len(predictionModels)
  
  for i in range(0, len(X)):
    x_val = np.load(X[i]) # loaded as RGB
#     for m_i in range(0, totalModels):
#       model = predictionModels[m_i]
#       x_val = preprocessData(x_val, m_i+base)
#       savePredictions(m_i+base, model.predict(x_val, verbose=1))      

In [0]:
def processPredictions(totalModels):
  
  Y_PREDS = np.zeros(totalModels*7*500)
  Y_PREDS = Y_PREDS.reshape(totalModels, 7, 500)

  for i in range(0, totalModels):
    Y_PREDS[i] = loadPredictions(i)
    
  for m_i in range(0, totalModels):
    Y_PREDS[m_i].reshape(1, 3500)
      
  K = 10
  topKPredictions = np.arrange(K*totalModels).reshape(totalModels,K)
  
  for k in range(1,K+1):
    for m_i in range(0, totalModels):
      topKPredictions[m_i, k-1] = top_k_accuracy(y_val_one_hot, Y_PREDS[m_i], k)
    
  return topKPredictions


# Run the tests

## 224x224 input models

In [0]:
imageRes = "224x224"

models = []
# models.append(vgg16_model)
models.append(vgg19_model)
models.append(resnet50_model)
models.append(densenet121_model)
models.append(densenet169_model)
models.append(densenet201_model)
 

savePredictions(predictionModels = models, imageRes = imageRes, base = 0)


## 299x299 input models

In [0]:
imageRes = "299x299"

models = []
models.append(inceptionv3_model)
models.append(inception_resnet_v2_model)
models.append(xception_model)

savePredictions(predictionModels = models, imageRes = imageRes, base = 6)


## 331x331 input models

In [0]:
imageRes = "331x331"

models = []
models.append(nasnet_model)

savePredictions(predictionModels = models, imageRes = imageRes, base = 9)

# Results and Analysis

In [0]:
modelName = ["" for x in range(10)]
modelName = { "VGG16", "VGG19", "RESNET50", "DENSENET121", "DENSENET169", "DENSENET201", "INCEPTIONV3", "INCEPTION_RESNET_V2", "XCEPTION", "NASNET" } 

topKPredictions = processPredictions(10)

out = ""
for k in range(0, 10):
  out = out + "Top-" + str(k) + ","
out = out + "\n"

for m_i in range(0, totalModels):
  for k in range(0, 10):
    out = out + topKPredictions[m_i, k] + ","
  out = out + "\n"

print(out)

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications.imagenet_utils import decode_predictions

# prepare the image for the VGG model
processed_image = densenet.preprocess_input(image_batch.copy())
 
# get the predicted probabilities for each class
predictions = densenet1.predict(processed_image)

# print predictions
# convert the probabilities to class labels
# We will get top 5 predictions which is the default
# plt.imshow(original)


for prediction in decode_predictions(predictions)[0]:
  print(prediction[1], prediction[2])